In [19]:
import pandas as pd
import os
import shutil
import numpy as np

# Data split

In [3]:
data = pd.read_csv('train.csv')

In [9]:
TEST_SIZE = 0.15
VAL_SIZE = 0.15
TRAIN_SIZE = 1. - TEST_SIZE - VAL_SIZE

In [12]:
p = np.array([TRAIN_SIZE, VAL_SIZE, TEST_SIZE])
folds = np.random.choice(3, len(data), p=p)

In [14]:
np.unique(folds, return_counts=True)

(array([0, 1, 2]), array([6964, 1482, 1466]))

In [16]:
train_data = data[folds == 0]
val_data = data[folds == 1]
test_data = data[folds == 2]

In [26]:
def save_fataset_fold(fold_data, fold_name, src_imgs_path, dataset_path):
    for file_id in fold_data['Id']:
        filename = file_id + '.jpg'
        
        src_pth = os.path.join(src_imgs_path, filename)
        dst_dir = os.path.join(dataset_path, fold_name)
        dst_pth = os.path.join(dst_dir, filename)
        
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copyfile(src_pth, dst_pth)
    
    pth = os.path.join(dataset_path, fold_name + '.csv')
    fold_data.to_csv(pth, index=None)

In [27]:
images_src_path = 'train'
dataset_path = 'torch_data'

os.makedirs(dataset_path, exist_ok=True)

folds = [
    ('train', train_data),
    ('val', val_data),
    ('test', test_data),
]
for fold_name, fold_data in folds:
    save_fataset_fold(
        fold_data=fold_data,
        fold_name=fold_name,
        src_imgs_path=images_src_path,
        dataset_path=dataset_path,
    )

# Test

In [42]:
for fold  in ['train', 'val', 'test']:
    imgs = os.listdir(f'torch_data/{fold}')
    ids = [e.strip('.jpg') for e in imgs]

    fold_data = pd.read_csv(f'torch_data/{fold}.csv')
    assert set(fold_data['Id']) == set(ids), 'images mistmatch'